In [1]:
#!pip install scikit-surprise

In [1]:
import os
from surprise import SVD
from surprise import SVDpp
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import BaselineOnly
from surprise import Reader
from surprise.model_selection import KFold
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
import pandas as pd

file_path = os.path.expanduser('ratings_small.csv')
movie_path = "movies_metadata.csv"
# friend_path = "friends.csv"

reader = Reader(line_format='user item rating timestamp', sep=',',skip_lines=1)
surprise_data = Dataset.load_from_file(file_path, reader=reader)

#Quantity of the recommond results
top_k = 5

movies_df = pd.read_csv(movie_path)

C:\Tools\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
# get similar users
def getSimilarUsers(top_k,u_id):
    all_trainset = surprise_data.build_full_trainset()
    algo = KNNBasic(k=40,min_k=3,sim_options={'user_based': True}) # sim_options={'name': 'cosine','user_based': True} cosine/msd/pearson/pearson_baseline
    algo.fit(all_trainset)
    user_inner_id = algo.trainset.to_inner_uid(u_id)
    user_neighbors = algo.get_neighbors(user_inner_id, k=top_k)
    user_neighbors = (algo.trainset.to_raw_uid(inner_id) for inner_id in user_neighbors)
    return user_neighbors
list(getSimilarUsers(5,'47'))

Computing the msd similarity matrix...
Done computing similarity matrix.


['12', '25', '53', '57', '96']

In [3]:
# get similar movies
def getSimilarItems(top_k,item_id):
    all_trainset = surprise_data.build_full_trainset()
    item_algo = KNNBasic(k=40,min_k=3,sim_options={'user_based': False}) # sim_options={'name': 'cosine','user_based': True} cosine/msd/pearson/pearson_baseline
    item_algo.fit(all_trainset)
    item_inner_id = item_algo.trainset.to_inner_iid(item_id)
    item_neighbors = item_algo.get_neighbors(item_inner_id, k=top_k)
    f_item_neighbors = (item_algo.trainset.to_raw_iid(inner_id)
                       for inner_id in item_neighbors)
    return f_item_neighbors
list(getSimilarItems(5,'710'))

Computing the msd similarity matrix...
Done computing similarity matrix.


['1339', '370', '552', '1378', '2174']

In [4]:
list(getSimilarItems(5,'110'))

Computing the msd similarity matrix...
Done computing similarity matrix.


['84236', '2903', '6598', '81158', '3865']

In [18]:
def getMoviesFromList(movieList):
    movies = []
    for movie in movieList:
        print(movies)
        for i in range(len(movies_df)):
            if movies_df['id'][i] == movie:
                movies.append(movies_df['title'][i])
                if movie not in movies_df['id']:
                    movies.append("Movie ID"+movies_df['id'][i]+"(currently unavailable)")
                
    return movies

In [19]:
def getMoviesFromTitle(title):
    #get movie ID or report error
    movies = []
    for i in range(len(movies_df)):
        if str(movies_df['title'][i]) == str(title):
            moviesList = list(getSimilarItems(top_k,movies_df['id'][i]))
            break
        elif i==len(movies_df)-1:
            return print("Couldn't find the movie, Please try again.")
    print(moviesList)
    return getMoviesFromList(moviesList)
    

getMoviesFromTitle("GoldenEye")


Computing the msd similarity matrix...
Done computing similarity matrix.
['1339', '370', '552', '1378', '2174']
[]
[]
[]
['Bread and Tulips', 'Movie ID552(currently unavailable)']
['Bread and Tulips', 'Movie ID552(currently unavailable)', 'Shortbus', 'Movie ID1378(currently unavailable)']


['Bread and Tulips',
 'Movie ID552(currently unavailable)',
 'Shortbus',
 'Movie ID1378(currently unavailable)']

In [21]:
def getMoviesFromUser(userID):
    user_df = pd.read_csv('ratings_small.csv')
    maxRate = 0
    for i in range(len(user_df)):
        if (str(user_df['userId'][i]) == str(userID)) and (float(user_df['rating'][i]) > maxRate):
            maxRate = user_df['rating'][i]
            likedID = user_df['movieId'][i]

    recList = list(getSimilarItems(top_k,str(likedID)))
    return getMoviesFromList(recList)

getMoviesFromUser('23')

111
Computing the msd similarity matrix...
Done computing similarity matrix.
['27369', '2046', '26', '1859', '3320']
[]
[]
['The Gift', 'Movie ID2046(currently unavailable)']
['The Gift', 'Movie ID2046(currently unavailable)', 'Walk on Water', 'Movie ID26(currently unavailable)']
['The Gift', 'Movie ID2046(currently unavailable)', 'Walk on Water', 'Movie ID26(currently unavailable)', 'Ninotchka', 'Movie ID1859(currently unavailable)']


['The Gift',
 'Movie ID2046(currently unavailable)',
 'Walk on Water',
 'Movie ID26(currently unavailable)',
 'Ninotchka',
 'Movie ID1859(currently unavailable)']